# Fitting data

In [ ]:
# Import packages
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import peaks as pks
import os

# Set default options
xr.set_options(cmap_sequential='Purples', keep_attrs=True)
%matplotlib inline
%config InlineBackend.figure_format='retina'

# Example data
from peaks.core.utils.sample_data import ExampleData

# For rendering output from iplot
from bokeh.io import output_notebook
output_notebook()

This tutorial gives a brief introduction to fitting data within `peaks`. `peaks` utilises :class:`lmfit` for fitting data, with some added or adapted methods to ease using this with data in a :class:`xarray.Dataarray` format, and some additional helper functions. The user should be familiar with :class:`lmfit` Models and methodoligies, see the `lmfit` [documentation](https://lmfit.github.io/lmfit-py/index.html).

:::{tip}
`peaks` wraps all standard :class:`lmfit.models` and provides some specific additional ones. These can be accessed at `peaks.core.fitting.models`. Any other compatible :class:`lmfit.model.Model` can be configured for use with `peaks` as

```python
from peaks.core.fitting.models import create_xarray_compatible_lmfit_model

# Make a compatible version of some lmfit-type `OriginalModel`
NewModel = create_xarray_compatible_lmfit_model(OriginalModel)
```
:::

:::{warning}
At present, `peaks` does not support fitting with units. The relevant :class:`xarray.DataArray`s are *dequantified* before fitting, and fit parameters should be specified without units.
:::

## General peak fitting
### Fitting 1D data
Let's generate some 1D data to fit, here a single MDC

In [ ]:
# Load a dispersion
disp = ExampleData.dispersion()
disp.plot()

In [ ]:
# Extract an MDC for initial analysis
cut1 = disp.MDC(105.06, 0.005).sel(theta_par=slice(0,15))#.pint.dequantify()
cut1.plot()

We'll fit this with the following model:

- 3 Lorentzian peaks
- Linear background
- Convolve with a Gaussian for experimental angular resolution

First we import the relevant base peak models and a model for performing the Gaussian convolution

In [ ]:
from peaks.core.fitting.models import LorentzianModel, LinearModel, GaussianConvolvedFitModel, ConstantModel

Now we build the model and initialise the parameters. The initial model is built using the `peaks`-wrapped versions of the standard models combined with binary operators (forming a :class:`lmfit.model.CompositeModel`), and then this is passed to the :function:`peaks.core.fitting.models.GaussianConvolvedFitModel` to add the Gaussian convolution.

In [ ]:
# Build the model
base_model = LinearModel(prefix="bg_")
for i in range(3):
    base_model += LorentzianModel(prefix=f"p{i}_")
model = GaussianConvolvedFitModel(base_model)

# Make the fit parameter list
params = model.make_params()

In [ ]:
# Show all the parameters
params

We can manually enter sensible starting parameters, e.g.:

In [ ]:
params = model.make_params(sigma_conv=dict(value=0.1, vary=False))  # e.g. fix the resolution 

In [ ]:
params

Or we can make use of the estimating capabilities of the individual models. This is not implemeted for all models, but is for most. It also does not work for the complete composite model in one go, and so we iterate through each peak and component in turn...

In [ ]:
# Estimate theta_par ranges for the peaks
p_range = {}
p_range[0] = [4,7.5]
p_range[1] = [7.5,9]
p_range[2] = [9,11]
pks.plot_grid([cut1.sel(theta_par=slice(p_range[i][0],p_range[i][1])) for i in range(3)])

In [ ]:
# Estimate fit parameters over these ranges
for i in range(3):
    t1,t2 = p_range[i]
    DC_data = cut1.sel(theta_par=slice(t1,t2))  # Select data over this range
    # With the `peaks` wrapped fit models, we pass the 1D DataArray directly instead of seperate y and x values
    peak_param_guess = LorentzianModel(prefix=f"p{i}_").guess(DC_data)  # Guess the initial params
    # Iterate through the guessed params and update the main params dictionary
    params.update(peak_param_guess)

# Make a guess for the background from the start of the range
DC_data = cut1.sel(theta_par=slice(0,4))
bg_param_guess = LinearModel(prefix="bg_").guess(DC_data)
params.update(bg_param_guess)

# Show updated params
params

We can plot the calcuated model using our guessed parameters on top of our data

In [ ]:
cut1.plot_fit_test(model=model, params=params, show_components=False)

This looks a reasonable starting point, so we can now fit the model. For this, we can make use of the :class:`peaks.core.fitting.fit` method, passing the data as an :class:`xarray.DataArray`. As this is a 1D DataArray, we do not need to explicitly specify the independent dimension (but we would if we had passed a higher dimensional array).

In [ ]:
fit_result = cut1.fit(model, params)

This returns a :class:`xarray.DataSet` which contains the results of the fit and standard errors as variables, as well as the full :class:`lmfit.model.ModelResult`.

In [ ]:
fit_result

In [ ]:
# Full lmfit ModelResult
fit_result['fit_model'].values[()]

#### Plotting fits
The fit, components and residual can be plot using the method `.plot_fit()`

In [ ]:
fit_result.plot_fit(figsize=(8,8))

#### Saving and loading fits
We can save the fit results (including the full model) and recover them using the `.save_fit` and `.load_fit` methods:

In [ ]:
fit_result.save_fit('fit_example')
previous_fit_result = pks.load_fit('fit_example')
if previous_fit_result == fit_result:
    print(True)

## Fitting multi-dimensional data
We'll now fit a series of MDCs from this dataset. We can either simply select the data over the required range we wish to fit, or for more complex fit regions, can mask the data:

In [ ]:
disp.plot()
# Define a custom mask region
mask_region = {'eV': [105.06, 105.04, 105.04, 105.06],
              'theta_par': [4, 6, 13, 13]}
# Note - choosing this region for illustration of the method, not because it is a particularly sensible region to fit over...!

# Plot the mask
pks.plot_ROI(mask_region, y='eV', linestyle='--')

# Select the data
data_to_fit = disp.mask_data(mask_region, return_integrated=False)

In [ ]:
data_to_fit.plot()

In [ ]:
data_to_fit

As we have now masked the data in a way that leaves NaNs at the edges for some of the data sets, we need to ensure the fit model can cope with this. To do this, we set the model `nan_policy`:

In [ ]:
model.nan_policy = 'omit'

For a 2D DataArray like this, we can choose to fit the data sequentially, where the result from the previous fit is used to initiate the starting parameters of the next fit, or using the same starting parameters for all fits (the latter also allows fitting the data using parallel processing). We will use the sequential mode here, which is the default when we are fitting 2D data.  

In [ ]:
# Fit the data - note we have reversed the direction in which to perform the sequential fit as we 
# figured out our initial guess parameters for the `last` MDC in this DataArray
%time fit_result2 = data_to_fit.fit(model, params, independent_var='theta_par', reverse_sequential_fit_order = True)

We can now plot the relevant results of the fit directly from the returned dataset.

In [ ]:
fig, ax = plt.subplots(figsize=(10,5), ncols=2)
data_to_fit.plot(ax=ax[0])
fit_result2['p0_center'].plot(ax=ax[0], y='eV', marker='o')
fit_result2['p1_center'].plot(ax=ax[0], y='eV', marker='o')
fit_result2['p2_center'].plot(ax=ax[0], y='eV', marker='o')

# Plot the fit with the error bars
ax[1].errorbar(x=fit_result2['p0_center'].data,
           y=fit_result2['p0_center'].eV.data,
           yerr=fit_result2['p0_center_stderr'].data,
        marker='o')

plt.tight_layout()

Calling the `plot_fit()` function on the fit result will now allow us to dynamically explore the fits and their residuals

In [ ]:
fit_result2.plot_fit()

### Dask arrays
The above methodology extends naturally to higher-dimensional data sets, although then sequential fitting cannot be used. If fitting higher-dimensional data (and sometimes also for lower-dimensional data), it is worth considering using :class:`dask`-backed :class:`xarray.DataArray`'s. This allows the fit to be performed in parallel, or in a lazy manner so that it is only computed when required. It is worth considering how best to `chunk` the DataArray to optimise for the intended fitting.

:::{tip}
Even though the `xarray.DataArray`s units are stripped during the fit function call, the `dask` graph still seems to retain some unit information, and so triggering the delayed computation can emit one or many `UnitStrippedWarning`s. There is a utility context manager (used below) to filter these warnings as a temporary workaround until we can solve this problem.
:::

Let's load a section of a spatial map:

In [ ]:
SM1 = (ExampleData.SM()  # File to load
       .isel(x2=slice(None,15))  # Select limited spatial range from the map
       .sel(theta_par=slice(0,5),eV=slice(74,76))  # Select limited angle and energy range
       .chunk(eV=-1,x1=1,x2=1)  # Ensure only a single chunk of the Dask array along the eV axis, but chunked in x1 and x2 
      )

In [ ]:
SM1

In [ ]:
# Extract a single EDC for illustration and setting up the fit
EDC_ = SM1.isel(x1=0,x2=0,theta_par=0)
EDC_.plot()

In [ ]:
fit_model = LorentzianModel() + ConstantModel()
params = fit_model.make_params(amplitude=20,
                              center=dict(value=75.25, min=74.5, max=75.7),
                              sigma=0.1,
                              c=dict(value=4, min=0)
                              )

In [ ]:
%time fit_results3 = SM1.fit(fit_model, params, independent_var='eV')

No fitting has been performed yet (note that this was extremely quick!), but we can still access the fit results plot, with the fits being performed as required

In [ ]:
fit_results3.plot_fit()

And we can make plots of relevant fit results

In [ ]:
from peaks.core.utils.misc import silence_unit_warnings
with silence_unit_warnings():
    fit_results3['center'].isel(theta_par=10).plot()  # This is the important part

## Fitting Au Fermi edge data

For fitting Au Fermi edge data, we employ a model :class:`peaks.core.fitting.LinearDosFermiModel` which employs a linear DOS, and Gaussian-broadened Fermi function, and a linear background above $E_F$ to account for potential detector non-linearities. This model can be imported and manually fit using the methods above, but we also have a utility method `fit_gold` which attempts an automated fitting to this model, and displays some key results as well as returning the actual fit result :class:`xarray.DataSet`.

In [ ]:
# Load a gold scan and restrict the range to a sensible amount
gold_scan = ExampleData.gold_reference4().sel(eV=slice(16.4,17),theta_par=slice(-10,10))
gold_scan.plot()

We can pass a single EDC:

In [ ]:
fit_result = gold_scan.isel(theta_par=0).fit_gold()

The fitted Fermi energy is also stored in the fit_result attributes

In [ ]:
fit_result.attrs

Or we can pass the measured angular-dependent Au reference, optionally specifying the type of function (polynomaial, average etc.) used to determine the Fermi level dependence on detector angle (via the `EF_correction_type` parameter; defaults to polynomail of order 4)

In [ ]:
fit_result = gold_scan.fit_gold()

The results of the `EF_correction` fit are stored as a dictionary in the fit results attribute

In [ ]:
fit_result.EF_correction


## Quick fitting

For simple functions (linear, polynomial of some degree, single peak + linear background), a quite fit can be performed by passing either a 1D DataArray, or a multi-dimensional array and specifying the independent variable. The quick fit is accessed via the `.quick_fit.MODEL` accessor, e.g.:

In [ ]:
example_DC = gold_scan.isel(theta_par=0).sel(eV=slice(None, 16.7))

In [ ]:
fit_result = example_DC.quick_fit.linear()

This returns the usual `peaks` :class:`xarray.DataSet` fit_result. The parameters can be seen directly: 

In [ ]:
fit_result

Or the plot can be returned as usual:

In [ ]:
fit_result.plot_fit()

This method scales to higher-dimensional data as for the full fit methods:

In [ ]:
SM1.quick_fit.lorentzian(independent_var='eV').plot_fit()